# Vendor Contract QA Agent Documentation POC

## Pre-requisites

In [1]:
%pip install duckduckgo-search beautifulsoup4 --quiet

Note: you may need to restart the kernel to use updated packages.


In [2]:
# load openai api key
import os

from dotenv import load_dotenv
load_dotenv()

if not 'OPENAI_API_KEY' in os.environ:
    raise ValueError('OPENAI_API_KEY is not set')

In [3]:
import sys

sys.path.append(os.getcwd())

from utils import (
    client,
    init_db,
    get_schema_description,
    get_tools_spec,
    show_json,
)

In [4]:
init_db()

# Create Agent

### Load Tools and DB Schema Spec

In [5]:
tools_spec = get_tools_spec()

Loaded 2 tool definitions
[
      {
            "type": "function",
            "function": {
                  "name": "query_database",
                  "description": "Query the contract database to retrieve information about contracts with a particular vendor",
                  "parameters": {
                        "type": "object",
                        "properties": {
                              "query": {
                                    "type": "string",
                                    "description": "SQL query to execute against the database"
                              }
                        },
                        "required": [
                              "query"
                        ]
                  }
            }
      },
      {
            "type": "function",
            "function": {
                  "name": "search_online",
                  "description": "Search the web for information",
                  "parameters": {
             

In [6]:
print(get_schema_description())

Table: vendors
  vendor_id: TEXT
  vendor_name: TEXT
  category: TEXT
  total_spend: INTEGER
  contracts: TEXT
Table: contracts
  contract_id: TEXT
  vendor_id: TEXT
  start_date: TEXT
  end_date: TEXT
  contract_value: INTEGER
  terms_conditions: TEXT
  products_description: TEXT



### Create an Assistant

In [7]:
AGENT_SYSTEM_PROMPT = f"""
# Mission:
You are an AI Agent that helps employees answer questions they might have about everything related to software vendors.
You will be asked questions such as "Do we have a vendor for cloud storage?" or "I need a tool for project management".
You should use the tools available to you as well as semantic search on the documents you have access to to answer these questions.

# Guidelines:
For "Do we have a vendor for cloud storage?", you could use the `query_database` to query the contracts database for cloud storage vendors.
Then you could search your document repository for information on the vendors you found.
If none are found, then you might search online using the `search_online` tool to discover new vendors.
Or, for the question "I need a tool for project management", if you cannot find a relevant vendor in the database,
  you could use the `search_online` tool to find out if any existing vendors provide project management tools.

# Constraints:
You should always try and find relevant information from the database.
You can search online to find new information or confirm information that you already know.
You should only fall back to your existing knowledge of vendors to help you come up with good search queries or when you want to enrich your answers.
  - For example, if the user is asking for a certain product and you find a vendor in the db that doesn't mention that product but you know they offer it, you can share that with the user.
You should only use your document retrieval system to find extra information related to vendors found in the database or online - essentially to enrich your knowledge before answering.
Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous.

# DB Schema:
{get_schema_description()}
""".strip()


# Create an agent using the OpenAI Assistants API
# agent = client.beta.assistants.create(
#     name="Vendor Contract Q/A Assistant",
#     instructions=AGENT_SYSTEM_PROMPT,
#     model="gpt-4-turbo",
#     tools=tools_spec,
# )
# show_json(agent)
# AGENT_ID = agent.id

# Or use an existing agent
AGENT_ID = "asst_sAavPqXvzJNzFtoUmtfF0qCj"

# Use the Agent

In [8]:
from utils import AgentEventHandler

os.environ["DEBUG"] = "0"

thread = client.beta.threads.create()


def send_message(input):
    client.beta.threads.messages.create(
        thread_id=thread.id,
        role="user",
        content=input["question"],
    )

    with client.beta.threads.runs.stream(
        thread_id=thread.id,
        assistant_id=AGENT_ID,
        event_handler=AgentEventHandler(input),
    ) as stream:
        stream.until_done()

    return input

In [9]:
input = {"question": "Do we have contracts with microsoft?"}
send_message(input)
input

> Querying database with:  {'query': "SELECT * FROM vendors WHERE vendor_name LIKE '%Microsoft%';"}
> Result:  [["V011", "Microsoft", "Productivity Software", 400000, "C014,C015"]]


> Querying database with:  {'query': "SELECT * FROM contracts WHERE contract_id IN ('C014', 'C015');"}
> Result:  [["C014", "V011", "2022-09-01", "2025-08-31", 210000, "Annual subscription for office productivity suite.", "Microsoft Office 365 suite including Word, Excel, PowerPoint, and Teams."], ["C015", "V011", "2023-10-01", "2026-09-30", 240000, "Azure compute and storage services. Pay-as-you-go pricing.", "Microsoft Azure services including virtual machines, storage accounts, and networking capabilities."]]


> Receiving message from agent:


HTML(value='\n<div id="message_container">\n\n</div>\n<style>\n#message_container {\n    padding: 10px;\n    b…

{'question': 'Do we have contracts with microsoft?',
 'tool_calls': [{'function': 'query_database',
   'arguments': {'query': "SELECT * FROM vendors WHERE vendor_name LIKE '%Microsoft%';"}},
  {'function': 'query_database',
   'arguments': {'query': "SELECT * FROM contracts WHERE contract_id IN ('C014', 'C015');"}}],
 'contexts': ['[["V011", "Microsoft", "Productivity Software", 400000, "C014,C015"]]',
  '[["C014", "V011", "2022-09-01", "2025-08-31", 210000, "Annual subscription for office productivity suite.", "Microsoft Office 365 suite including Word, Excel, PowerPoint, and Teams."], ["C015", "V011", "2023-10-01", "2026-09-30", 240000, "Azure compute and storage services. Pay-as-you-go pricing.", "Microsoft Azure services including virtual machines, storage accounts, and networking capabilities."]]'],
 'messages': ['Yes, we have contracts with Microsoft. Here are the details of the contracts:\n\n1. **Microsoft Office 365 Suite Contract (C014)**\n   - **Start Date:** 2022-09-01\n   -

In [13]:
import pandas as pd
import validmind as vm

vm.init()

vm_model = vm.init_model(predict_fn=send_message, input_id="vendor_qa_agent")
vm_model.predict(pd.DataFrame({"question": ["Do we have contracts with microsoft?"]}))

2024-05-10 17:38:09,824 - INFO(validmind.api_client): Connected to ValidMind. Project: [Demo] Customer Churn Model - Initial Validation (clnt1f4qc00ap15lfts8ur7lw)


> Querying database with:  {'query': "SELECT * FROM vendors WHERE vendor_name LIKE '%Microsoft%';"}
> Result:  [["V011", "Microsoft", "Productivity Software", 400000, "C014,C015"]]


> Querying database with:  {'query': "SELECT * FROM contracts WHERE contract_id IN ('C014', 'C015');"}
> Result:  [["C014", "V011", "2022-09-01", "2025-08-31", 210000, "Annual subscription for office productivity suite.", "Microsoft Office 365 suite including Word, Excel, PowerPoint, and Teams."], ["C015", "V011", "2023-10-01", "2026-09-30", 240000, "Azure compute and storage services. Pay-as-you-go pricing.", "Microsoft Azure services including virtual machines, storage accounts, and networking capabilities."]]


> Receiving message from agent:


HTML(value='\n<div id="message_container">\n\n</div>\n<style>\n#message_container {\n    padding: 10px;\n    b…

[{'question': 'Do we have contracts with microsoft?',
  'tool_calls': [{'function': 'query_database',
    'arguments': {'query': "SELECT * FROM vendors WHERE vendor_name LIKE '%Microsoft%';"}},
   {'function': 'query_database',
    'arguments': {'query': "SELECT * FROM contracts WHERE contract_id IN ('C014', 'C015');"}}],
  'contexts': ['[["V011", "Microsoft", "Productivity Software", 400000, "C014,C015"]]',
   '[["C014", "V011", "2022-09-01", "2025-08-31", 210000, "Annual subscription for office productivity suite.", "Microsoft Office 365 suite including Word, Excel, PowerPoint, and Teams."], ["C015", "V011", "2023-10-01", "2026-09-30", 240000, "Azure compute and storage services. Pay-as-you-go pricing.", "Microsoft Azure services including virtual machines, storage accounts, and networking capabilities."]]'],
  'messages': ['Yes, we have contracts with Microsoft. Here are the details of the contracts:\n\n1. **Microsoft Office 365 Suite Contract (C014)**\n   - **Start Date:** 2022-09-

In [27]:
test_df = pd.DataFrame({
    "question": [
        "Do we have contracts with microsoft?",
        "Do we have contracts with google?",
        "Do we have contracts with amazon?",
        "What vendors do we have that offer cloud storage?",
        "Do we have relationships with server hardware vendors?",
        "How much is our total spend on project management software?",
        "I need an ERP system for our company. Can you help me find one?",
    ],
    "ground_truth": [
        "Yes, we have 2 contracts with Microsoft: C014 and C015.",
        "No, we do not have contracts with Google.",
        "Yes, we have 2 contracts with Amazon: C007 and C008.",
        "We have multiple vendors who have a cloud storage offering: Amazon Web Services (Vendor ID: V005), Microsoft (Vendor ID: V011), IBM (Vendor ID: V012) and Oracle (Vendor ID: V014)",
        "We have relationships with the following server hardware vendors: Dell (Vendor ID: V013), HP (Vendor ID: V015) and Cisco (Vendor ID: V001).",
        "We don't have any existing contracts for project management software. So the total spend is $0.",
        "We have an existing relationship with SAP for ERP software (Vendor ID: V004). The following two contracts are in place: C005 and C006.",
    ],
})
test_df["prediction"] = vm_model.predict(test_df)

> Querying database with:  {'query': "SELECT * FROM vendors WHERE vendor_name LIKE '%Microsoft%';"}
> Result:  [["V011", "Microsoft", "Productivity Software", 400000, "C014,C015"]]


> Querying database with:  {'query': "SELECT * FROM contracts WHERE contract_id IN ('C014', 'C015');"}
> Result:  [["C014", "V011", "2022-09-01", "2025-08-31", 210000, "Annual subscription for office productivity suite.", "Microsoft Office 365 suite including Word, Excel, PowerPoint, and Teams."], ["C015", "V011", "2023-10-01", "2026-09-30", 240000, "Azure compute and storage services. Pay-as-you-go pricing.", "Microsoft Azure services including virtual machines, storage accounts, and networking capabilities."]]


> Receiving message from agent:


HTML(value='\n<div id="message_container">\n\n</div>\n<style>\n#message_container {\n    padding: 10px;\n    b…

> Querying database with:  {'query': "SELECT * FROM vendors WHERE vendor_name LIKE '%Google%';"}
> Result:  []


> Receiving message from agent:


HTML(value='\n<div id="message_container">\n\n</div>\n<style>\n#message_container {\n    padding: 10px;\n    b…

> Querying database with:  {'query': "SELECT * FROM vendors WHERE vendor_name LIKE '%Amazon%';"}
> Result:  [["V005", "Amazon Web Services", "Cloud Computing", 215000, "C007,C008"]]


> Querying database with:  {'query': "SELECT * FROM contracts WHERE contract_id IN ('C007', 'C008');"}
> Result:  [["C007", "V005", "2022-08-01", "2025-07-31", 120000, "Cloud services provisioning - unlimited consulting hours to setup and provision new AWS services for 3 years - renewal option available.", "Amazon Web Services (AWS) cloud services provision."], ["C008", "V005", "2021-09-01", "2024-08-31", 95000, "Pay-as-you-go AWS cloud services.", "Amazon Web Services (AWS) cloud services"]]


> Receiving message from agent:


...

HTML(value='\n<div id="message_container">\n\n</div>\n<style>\n#message_container {\n    padding: 10px;\n    b…

> Querying database with:  {'query': "SELECT SUM(total_spend) FROM vendors WHERE category LIKE '%Project Management%';"}
> Result:  [[null]]


> Receiving message from agent:


HTML(value='\n<div id="message_container">\n\n</div>\n<style>\n#message_container {\n    padding: 10px;\n    b…

> Querying database with:  {'query': "SELECT * FROM vendors WHERE category LIKE '%ERP%' OR category LIKE '%Enterprise Resource Planning%';"}
> Result:  [["V004", "SAP", "Enterprise Resource Planning", 450000, "C005,C006"]]


> Receiving message from agent:


HTML(value='\n<div id="message_container">\n\n</div>\n<style>\n#message_container {\n    padding: 10px;\n    b…

In [28]:
test_df.head()

,question,ground_truth,prediction
0,Do we have contracts with microsoft?,"Yes, we have 2 contracts with Microsoft: C014 ...",{'question': 'Do we have contracts with micros...
1,Do we have contracts with google?,"No, we do not have contracts with Google.",{'question': 'Do we have contracts with google...
2,Do we have contracts with amazon?,"Yes, we have 2 contracts with Amazon: C007 and...",{'question': 'Do we have contracts with amazon...
3,What vendors do we have that offer cloud storage?,We have multiple vendors who have a cloud stor...,{'question': 'What vendors do we have that off...
4,Do we have relationships with server hardware ...,We have relationships with the following serve...,{'question': 'Do we have relationships with se...


In [29]:
import json

print(json.dumps(test_df["prediction"][0], indent=5))

{
     "question": "Do we have contracts with microsoft?",
     "ground_truth": "Yes, we have 2 contracts with Microsoft: C014 and C015.",
     "tool_calls": [
          {
               "function": "query_database",
               "arguments": {
                    "query": "SELECT * FROM vendors WHERE vendor_name LIKE '%Microsoft%';"
               }
          },
          {
               "function": "query_database",
               "arguments": {
                    "query": "SELECT * FROM contracts WHERE contract_id IN ('C014', 'C015');"
               }
          }
     ],
     "contexts": [
          "[[\"V011\", \"Microsoft\", \"Productivity Software\", 400000, \"C014,C015\"]]",
          "[[\"C014\", \"V011\", \"2022-09-01\", \"2025-08-31\", 210000, \"Annual subscription for office productivity suite.\", \"Microsoft Office 365 suite including Word, Excel, PowerPoint, and Teams.\"], [\"C015\", \"V011\", \"2023-10-01\", \"2026-09-30\", 240000, \"Azure compute and storage service

In [30]:
test_df["answer"] = test_df["prediction"].apply(lambda x: x["messages"][0])

In [31]:
test_df["contexts"] = test_df["prediction"].apply(lambda x: x["contexts"])

In [37]:
test_df.head()

,question,ground_truth,prediction,answer,contexts
0,Do we have contracts with microsoft?,"Yes, we have 2 contracts with Microsoft: C014 ...",{'question': 'Do we have contracts with micros...,"Yes, we have active contracts with Microsoft. ...","[[[""V011"", ""Microsoft"", ""Productivity Software..."
1,Do we have contracts with google?,"No, we do not have contracts with Google.",{'question': 'Do we have contracts with google...,We do not currently have any contracts with Go...,[[]]
2,Do we have contracts with amazon?,"Yes, we have 2 contracts with Amazon: C007 and...",{'question': 'Do we have contracts with amazon...,"Yes, we have active contracts with Amazon Web ...","[[[""V005"", ""Amazon Web Services"", ""Cloud Compu..."
3,What vendors do we have that offer cloud storage?,We have multiple vendors who have a cloud stor...,{'question': 'What vendors do we have that off...,We have contracts with the following vendors t...,"[[[""V005"", ""Amazon Web Services"", ""Cloud Compu..."
4,Do we have relationships with server hardware ...,We have relationships with the following serve...,{'question': 'Do we have relationships with se...,"Yes, we have relationships with the following ...","[[[""V001"", ""Cisco Systems"", ""Networking Hardwa..."


In [38]:
vm_dataset = vm.init_dataset(test_df, input_id="agent_test_dataset")

2024-05-10 18:00:49,586 - INFO(validmind.client): Pandas dataset detected. Initializing VM Dataset instance...


In [39]:
vm.tests.list_tests(filter="rag")

Test Type,Name,Description,ID
Metric,Answer Correctness,Evaluates the correctness of answers in a dataset with respect to the provided ground truths and...,validmind.model_validation.ragas.AnswerCorrectness
Metric,Answer Relevance,"Evaluates the relevance of answers in a dataset with respect to the provided questions and contexts,...",validmind.model_validation.ragas.AnswerRelevance
Metric,Context Relevancy,Evaluates the context relevancy metric for entries in a dataset and visualizes the results....,validmind.model_validation.ragas.ContextRelevancy
Metric,Context Entity Recall,Evaluates the context entity recall metric for dataset entries and visualizes the results....,validmind.model_validation.ragas.ContextEntityRecall
Metric,Aspect Critique,Evaluates the harmfulness of answers in a dataset and visualizes the results in a histogram....,validmind.model_validation.ragas.AspectCritique
Metric,Answer Similarity,Calculates the answer similarity metric for dataset entries based on a provided ground truth....,validmind.model_validation.ragas.AnswerSimilarity
Metric,Context Precision,Evaluates the context precision metric for dataset entries and visualizes the results....,validmind.model_validation.ragas.ContextPrecision
Metric,Faithfulness,Evaluates the faithfulness metric for generated answers in a dataset and visualizes the results....,validmind.model_validation.ragas.Faithfulness
Metric,Context Recall,Evaluates the context recall metric for dataset entries and visualizes the results in a histogram....,validmind.model_validation.ragas.ContextRecall


In [41]:
for test_id in vm.tests.list_tests(filter="rag", pretty=False):
    vm.tests.run_test(test_id, inputs={"dataset": vm_dataset})

Evaluating:   0%|          | 0/7 [00:00<?, ?it/s]

promote has been superseded by promote_options='default'.
promote has been superseded by promote_options='default'.


Evaluating:   0%|          | 0/7 [00:00<?, ?it/s]

/Users/jwalz/Library/Caches/pypoetry/virtualenvs/validmind-mI3jzOkk-py3.11/lib/python3.11/site-packages/datasets/table.py:1395: FutureWarning:

promote has been superseded by promote_options='default'.

/Users/jwalz/Library/Caches/pypoetry/virtualenvs/validmind-mI3jzOkk-py3.11/lib/python3.11/site-packages/datasets/table.py:1421: FutureWarning:

promote has been superseded by promote_options='default'.



Evaluating:   0%|          | 0/7 [00:00<?, ?it/s]

...

/Users/jwalz/Library/Caches/pypoetry/virtualenvs/validmind-mI3jzOkk-py3.11/lib/python3.11/site-packages/datasets/table.py:1395: FutureWarning:

promote has been superseded by promote_options='default'.

/Users/jwalz/Library/Caches/pypoetry/virtualenvs/validmind-mI3jzOkk-py3.11/lib/python3.11/site-packages/datasets/table.py:1421: FutureWarning:

promote has been superseded by promote_options='default'.



Evaluating:   0%|          | 0/7 [00:00<?, ?it/s]

/Users/jwalz/Library/Caches/pypoetry/virtualenvs/validmind-mI3jzOkk-py3.11/lib/python3.11/site-packages/datasets/table.py:1395: FutureWarning:

promote has been superseded by promote_options='default'.

/Users/jwalz/Library/Caches/pypoetry/virtualenvs/validmind-mI3jzOkk-py3.11/lib/python3.11/site-packages/datasets/table.py:1421: FutureWarning:

promote has been superseded by promote_options='default'.



Evaluating:   0%|          | 0/7 [00:00<?, ?it/s]

/Users/jwalz/Library/Caches/pypoetry/virtualenvs/validmind-mI3jzOkk-py3.11/lib/python3.11/site-packages/datasets/table.py:1395: FutureWarning:

promote has been superseded by promote_options='default'.

/Users/jwalz/Library/Caches/pypoetry/virtualenvs/validmind-mI3jzOkk-py3.11/lib/python3.11/site-packages/datasets/table.py:1421: FutureWarning:

promote has been superseded by promote_options='default'.

